In [2]:
import findspark
findspark.init('/home/mint/spark-2.1.0-bin-hadoop2.7')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tree').getOrCreate()
data = spark.read.csv('College.csv', inferSchema=True, header=True)
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [3]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols = ['Apps', 'Accept', 'Enroll', 'Top10perc', 'Top25perc', "F_Undergrad",
                                         'P_Undergrad', 'Outstate', 'Room_Board', 'Books', 'Personal', 'PhD',
                                         'Terminal', 'S_F_Ratio', 'perc_alumni', 'Expend', 'Grad_Rate'],
                            outputCol = 'features')
output = assembler.transform(data)

In [4]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol = 'Private', outputCol = 'PrivateIndex')
output_fixed = indexer.fit(output).transform(output)
# output_fixed.show()

In [5]:
final_data = output_fixed.select('features', 'PrivateIndex')

In [6]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml import Pipeline
dtc = DecisionTreeClassifier(labelCol = 'PrivateIndex', featuresCol = 'features')
rfc = RandomForestClassifier(labelCol = 'PrivateIndex', featuresCol = 'features')
gbt = GBTClassifier(labelCol = 'PrivateIndex', featuresCol = 'features')

In [7]:
dtc_model = dtc.fit(train_data)
dtc_preds = dtc_model.transform(test_data)
rfc_model = rfc.fit(train_data)
rfc_preds = rfc_model.transform(test_data)
gbt_model = gbt.fit(train_data)
gbt_preds = gbt_model.transform(test_data)

In [11]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')
print('DTC')
print(my_binary_eval.evaluate(dtc_preds))
print('RFC')
print(my_binary_eval.evaluate(rfc_preds))
print('GBT')
print(BinaryClassificationEvaluator(labelCol='PrivateIndex', rawPredictionCol='prediction').evaluate(gbt_preds))

DTC
0.8635129490392649
RFC
0.9701336675020885
GBT
0.851921470342523


In [17]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex', metricName='accuracy')
print('DTC')
print(acc_eval.evaluate(dtc_preds))
print('RF')
print(acc_eval.evaluate(rfc_preds))
print('DTC')
print(acc_eval.evaluate(gbt_preds))

DTC
0.8976744186046511
RF
0.9209302325581395
DTC
0.9023255813953488
